## ETL

In [49]:
import pandas as pd
import re
import datetime
import numpy as np

Extraccion

In [50]:
# Extraemos los datos de las horas que necesitamos
hechos = pd.read_excel('../Data/homicidios.xlsx', sheet_name='HECHOS')
victimas = pd.read_excel('../Data/homicidios.xlsx', sheet_name='VICTIMAS')

## Comenzamos con el dataset de hechos.

In [51]:
hechos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     696 non-null    object        
 1   N_VICTIMAS             696 non-null    int64         
 2   FECHA                  696 non-null    datetime64[ns]
 3   AAAA                   696 non-null    int64         
 4   MM                     696 non-null    int64         
 5   DD                     696 non-null    int64         
 6   HORA                   696 non-null    object        
 7   HH                     696 non-null    object        
 8   LUGAR_DEL_HECHO        696 non-null    object        
 9   TIPO_DE_CALLE          696 non-null    object        
 10  Calle                  695 non-null    object        
 11  Altura                 129 non-null    float64       
 12  Cruce                  525 non-null    object        
 13  Direc

Tiene pocos registros nulos, pero si tiene columnas con muchos nulos

In [52]:
# Vemos como estan conformados los registros
hechos.head()

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,2034.0,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,NaN,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,SD
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,NaN,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS


In [53]:
# Aca vemos que hay cruces donde ocurren mas accidentes
hechos.Cruce.value_counts()

Cruce
PAZ, GRAL. AV.              14
ESCALADA AV.                 9
RIVADAVIA AV.                8
INDEPENDENCIA AV.            7
BALBIN, RICARDO, DR. AV.     6
                            ..
LA RIOJA                     1
TRAFUL                       1
HUMAITA                      1
MANZANARES                   1
HUGO, VICTOR                 1
Name: count, Length: 317, dtype: int64

In [54]:
# Capitalizamos todas las columnas
hechos.columns = hechos.columns.str.capitalize()

In [55]:
# Vemos la cantidad de columnas que tenemos
hechos.columns.tolist()

['Id',
 'N_victimas',
 'Fecha',
 'Aaaa',
 'Mm',
 'Dd',
 'Hora',
 'Hh',
 'Lugar_del_hecho',
 'Tipo_de_calle',
 'Calle',
 'Altura',
 'Cruce',
 'Dirección normalizada',
 'Comuna',
 'Xy (caba)',
 'Pos x',
 'Pos y',
 'Participantes',
 'Victima',
 'Acusado']

Ahora procedemos a analizar las fechas

In [56]:
fechas_correctas = [] # Instanciamos una lista vacia de fechas correctas
fechas_incorrectas = [] # Instanciamos una lista vacia de fechas incorrectas

for fecha in hechos.Fecha.unique().tolist(): # Recorremos las fechas de la columna fecha
    fecha = str(fecha) # Transformamos a string
    try:
        # Intenta analizar la fecha
        datetime.datetime.strptime(fecha, '%Y-%m-%d %H:%M:%S') # Si la fecha tiene el formato correcto
        fechas_correctas.append(fecha) # La agregamos a las fechas correctas
    except ValueError:
        fechas_incorrectas.append(fecha) # Si tiene el formato incorrecto, evade el error y suma a incorrectas

# Imprimimos los resultados

print("Fechas correctas:") 
for fecha in fechas_correctas:
    print(fecha)

print("\nFechas incorrectas:")
for fecha in fechas_incorrectas:
    print(fecha)

Fechas correctas:
2016-01-01 00:00:00
2016-01-02 00:00:00
2016-01-03 00:00:00
2016-01-10 00:00:00
2016-01-21 00:00:00
2016-01-24 00:00:00
2016-01-29 00:00:00
2016-02-08 00:00:00
2016-02-10 00:00:00
2016-02-14 00:00:00
2016-02-15 00:00:00
2016-02-17 00:00:00
2016-02-21 00:00:00
2016-02-28 00:00:00
2016-03-02 00:00:00
2016-03-04 00:00:00
2016-03-08 00:00:00
2016-03-12 00:00:00
2016-03-13 00:00:00
2016-03-14 00:00:00
2016-03-19 00:00:00
2016-03-21 00:00:00
2016-03-23 00:00:00
2016-03-29 00:00:00
2016-03-30 00:00:00
2016-03-31 00:00:00
2016-04-11 00:00:00
2016-04-15 00:00:00
2016-04-17 00:00:00
2016-04-19 00:00:00
2016-04-20 00:00:00
2016-04-22 00:00:00
2016-04-23 00:00:00
2016-04-25 00:00:00
2016-04-26 00:00:00
2016-05-04 00:00:00
2016-05-08 00:00:00
2016-05-14 00:00:00
2016-05-20 00:00:00
2016-05-21 00:00:00
2016-05-22 00:00:00
2016-05-23 00:00:00
2016-06-11 00:00:00
2016-06-12 00:00:00
2016-06-13 00:00:00
2016-06-16 00:00:00
2016-06-18 00:00:00
2016-06-22 00:00:00
2016-06-27 00:00:00
20

Este analisis nos dice que no tenemos ninguna fecha incorrecta asi que podemos eliminar las columnas de 'Aaaa','Mm' y 'Dd'

In [57]:
cols_a_eliminar = [
    'Xy (caba)', # Eliminamos esta columna ya que tenemos las coordenadas en otras columnas
    'Aaaa',
    'Mm',
    'Dd',
    'Calle', # La calle la tenemos en la direccion normalizada
    'Altura', # La altura la tenemos en la direccion normalizada
    'Participantes' # Eliminamos los participantes porque los tenemos en victimas y acusados
]

hechos = hechos.drop(columns=cols_a_eliminar) # Eliminamos las columnas

Elijo cuales capitalizar y cuales minusculizar, dependiendo del dato.

In [58]:
a_cap = ['Cruce','Dirección normalizada', 'Lugar_del_hecho'] # A capitalizar
a_min = ['Tipo_de_calle','Victima', 'Acusado'] # A minuscula

In [59]:
for columna in a_cap:
    hechos[columna] = hechos[columna].str.capitalize()

# Hacemos las capidalizaciones y las minusculas

for columna in a_min:
    hechos[columna] = hechos[columna].str.lower()

In [60]:
# LLeno las direcciones normalizadas nulas con su respectivo dato de Lugar_del_hecho
hechos['Dirección normalizada'] = hechos['Dirección normalizada'].fillna(hechos['Lugar_del_hecho'])
hechos.drop('Lugar_del_hecho',axis=1, inplace=True) # Elimino la columna Lugar_del_hecho

In [13]:
hechos.rename(columns={'Dirección normalizada':'Lugar_del_hecho'}, inplace=True) # Renombro la columna a gusto

In [14]:
hechos['Hora'] = hechos['Hora'].apply(lambda x: str(x)[:2]) # Creo la columna hora con la hora del suceso
hechos.drop('Hh',axis=1, inplace=True) # Y elimino 'Hh'

In [15]:
for col in ['N_victimas','Tipo_de_calle', 'Comuna', 'Victima', 'Acusado']:
    print(hechos[col].unique().tolist()) # Visualizo los valore unicos por columna

[1, 2, 3]
['avenida', 'gral paz', 'calle', 'autopista']
[8, 9, 1, 11, 15, 4, 7, 12, 3, 13, 14, 10, 6, 2, 5, 0]
['moto', 'auto', 'peaton', 'sd', 'cargas', 'bicicleta', 'pasajeros', 'movil', 'objeto fijo', 'peaton_moto']
['auto', 'pasajeros', 'sd', 'objeto fijo', 'cargas', 'moto', 'multiple', 'otro', 'bicicleta', 'tren']


Vemos que tenemos ciertos registros 'SD' o, sin dato, asi que los imputamos

In [16]:
hechos.loc[hechos['Hora'] == 'SD', 'Hora'] = '00' # Las horas sin dato las imputo como las 00hs
hechos.Hora = hechos.Hora.astype(int) # Transformo la columna a enteros

Imputo los sindato a nan

In [17]:
def sd_to_null(text): # Creamos la funcion
    if type(text) == str: # Si el dato es de tipo string
        if text.lower() == 'sd': # Entonces lo pasamos a minuscula para contener [Sd, SD o sD]
            return np.nan # Y si es sin dato, lo cambiamos a nulo
        else:
            return text # Sino, retornamos el texto
    return text # Sino, retornamos el texto

In [18]:
for col in hechos.columns.tolist():
    hechos[col]=hechos[col].apply(lambda x: sd_to_null(x)) # Aplicamos a cada columna

Listo, nulos y sin datos, tratados

In [19]:
hechos[hechos['Pos x'] == '.'].count() # Las latitudes y longitudes tienen puntos en vez de nulos

Id                 12
N_victimas         12
Fecha              12
Hora               12
Tipo_de_calle      12
Cruce               0
Lugar_del_hecho    11
Comuna             12
Pos x              12
Pos y              12
Victima            11
Acusado             9
dtype: int64

In [20]:
hechos[hechos['Pos y'] == '.'].count()

Id                 12
N_victimas         12
Fecha              12
Hora               12
Tipo_de_calle      12
Cruce               0
Lugar_del_hecho    11
Comuna             12
Pos x              12
Pos y              12
Victima            11
Acusado             9
dtype: int64

Tenemos solo 12 registros, pero no logro imputarlos, ya que por ejemplo, Autopista lugones pk 10000, en el maps no permite buscar por kilometro en esa autopista.

In [21]:
hechos[(hechos['Pos y'] == '.') & (hechos['Pos x'] == '.')][['Pos y','Pos x']] # Mostramos los registros

,Pos y,Pos x
38,.,.
106,.,.
119,.,.
139,.,.
176,.,.
180,.,.
181,.,.
256,.,.
313,.,.
546,.,.


In [22]:
hechos.loc[hechos['Pos y'] == '.', 'Pos y'] = np.nan # Transformo los puntos a nulos en ambas columnas
hechos.loc[hechos['Pos x'] == '.', 'Pos x'] = np.nan

In [23]:
hechos['Pos x'] = hechos['Pos x'].astype(float) # Y ahora los registros a flotantes
hechos['Pos y'] = hechos['Pos y'].astype(float) # Y ahora los registros a flotantes

Renombre de columnas

In [24]:
# Renombro las columnas para una mejor comprension
hechos.rename(columns={'Id':'Id_hecho',
 'N_victimas':'Victimas',
 'Fecha':'Fecha',
 'Hora':'Hora',
 'Tipo_de_calle':'Tipo_calle',
 'Cruce':'Cruce',
 'Lugar_del_hecho':'Direccion',
 'Comuna':'Comuna',
 'Pos x': 'Longitud',
 'Pos y': 'Latitud',
 'Victima':'Tipo_victima',
 'Acusado':'Tipo_acusado'}, inplace=True)

Analisis de valores repetidos

In [25]:
for col in hechos.columns.tolist():
    print(col,":",hechos[col].duplicated().unique()) # Vemos los unicos

Id_hecho : [False]
Victimas : [False  True]
Fecha : [False  True]
Hora : [False  True]
Tipo_calle : [False  True]
Cruce : [False  True]
Direccion : [False  True]
Comuna : [False  True]
Longitud : [False  True]
Latitud : [False  True]
Tipo_victima : [False  True]
Tipo_acusado : [False  True]


Vemos que tenemos duplicados en casi todas las columnas, vamos a analizarlos

In [26]:
hechos[hechos[['Longitud','Latitud']].duplicated()][['Longitud','Latitud']]

,Longitud,Latitud
7,-58.508775,-34.669777
37,-58.406239,-34.650765
68,-58.467435,-34.534769
89,-58.444513,-34.684759
106,NaN,NaN
...,...,...
670,-58.393039,-34.598711
672,-58.500738,-34.549795
676,-58.427797,-34.622073
690,-58.405969,-34.610120


Primero vemos que las coordenadas no estan realmente duplicadas, la funcion duplicated() las toma como duplicadas xq varian por decimales muy chicos.

In [27]:
hechos[hechos['Direccion'].duplicated()].head()

,Id_hecho,Victimas,Fecha,Hora,Tipo_calle,Cruce,Direccion,Comuna,Longitud,Latitud,Tipo_victima,Tipo_acusado
7,2016-0010,1,2016-01-29,15,gral paz,De los corrales av.,"Paz, gral. av. y de los corrales av.",9,-58.508775,-34.669777,moto,auto
37,2016-0051,1,2016-04-20,4,avenida,"Bonavena, oscar natalio","Alcorta, amancio av. y bonavena, oscar natalio",4,-58.406239,-34.650765,moto,cargas
68,2016-0092,1,2016-07-15,11,gral paz,Del libertador av.,"Paz, gral. av. y del libertador av.",13,-58.467435,-34.534769,moto,cargas
89,2016-0116,1,2016-09-02,20,avenida,Escalada av.,27 de febrero av. y escalada av.,8,-58.444513,-34.684759,moto,auto
112,2016-0143,1,2016-11-06,4,avenida,Escalada av.,27 de febrero av. y escalada av.,8,-58.444513,-34.684759,moto,NaN


Vemos que pese a q haya direcciones y cruces duplicadas, es porque se ve que en esas direcciones o cruces suceden siniestros mas frecuentemente. Realmente no es un error que haya duplicados

## Ahora trabajamos con las victimas

In [28]:
victimas.info() # Veamos su informacion

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID_hecho             717 non-null    object        
 1   FECHA                717 non-null    datetime64[ns]
 2   AAAA                 717 non-null    int64         
 3   MM                   717 non-null    int64         
 4   DD                   717 non-null    int64         
 5   ROL                  717 non-null    object        
 6   VICTIMA              717 non-null    object        
 7   SEXO                 717 non-null    object        
 8   EDAD                 717 non-null    object        
 9   FECHA_FALLECIMIENTO  717 non-null    object        
dtypes: datetime64[ns](1), int64(3), object(6)
memory usage: 56.1+ KB


De primeras, no tenemos ningun nulo.

In [29]:
victimas.head()

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03 00:00:00
3,2016-0004,2016-01-10,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18,SD
4,2016-0005,2016-01-21,2016,1,21,CONDUCTOR,MOTO,MASCULINO,29,2016-02-01 00:00:00


In [30]:
victimas.columns = victimas.columns.str.capitalize() # Capitalizo todas las columnas

Hago el mismo analisis anterior de fechas para ambas columnas

In [31]:
# Para 'Fecha'
fechas_correctas = []
fechas_incorrectas = []

for fecha in victimas.Fecha.unique().tolist():
    fecha = str(fecha)
    try:
        # Intenta analizar la fecha
        datetime.datetime.strptime(fecha, '%Y-%m-%d %H:%M:%S')
        fechas_correctas.append(fecha)
    except ValueError:
        # Captura las excepciones ValueError (formato incorrecto)
        fechas_incorrectas.append(fecha)

print("Fechas correctas:")
for fecha in fechas_correctas:
    print(fecha)

print("\nFechas incorrectas:")
for fecha in fechas_incorrectas:
    print(fecha)

Fechas correctas:
2016-01-01 00:00:00
2016-01-02 00:00:00
2016-01-03 00:00:00
2016-01-10 00:00:00
2016-01-21 00:00:00
2016-01-24 00:00:00
2016-01-29 00:00:00
2016-02-08 00:00:00
2016-02-10 00:00:00
2016-02-14 00:00:00
2016-02-15 00:00:00
2016-02-17 00:00:00
2016-02-21 00:00:00
2016-02-28 00:00:00
2016-03-02 00:00:00
2016-03-04 00:00:00
2016-03-08 00:00:00
2016-03-12 00:00:00
2016-03-13 00:00:00
2016-03-14 00:00:00
2016-03-19 00:00:00
2016-03-21 00:00:00
2016-03-23 00:00:00
2016-03-29 00:00:00
2016-03-30 00:00:00
2016-03-31 00:00:00
2016-04-11 00:00:00
2016-04-15 00:00:00
2016-04-17 00:00:00
2016-04-19 00:00:00
2016-04-20 00:00:00
2016-04-22 00:00:00
2016-04-23 00:00:00
2016-04-25 00:00:00
2016-04-26 00:00:00
2016-05-04 00:00:00
2016-05-08 00:00:00
2016-05-14 00:00:00
2016-05-20 00:00:00
2016-05-21 00:00:00
2016-05-22 00:00:00
2016-05-23 00:00:00
2016-06-11 00:00:00
2016-06-12 00:00:00
2016-06-13 00:00:00
2016-06-16 00:00:00
2016-06-18 00:00:00
2016-06-22 00:00:00
2016-06-27 00:00:00
20

No tenemos fechas en formato incorrecto.

In [32]:
# Para 'Fecha_fallecimiento'
fechas_correctas = []
fechas_incorrectas = []

for fecha in victimas.Fecha_fallecimiento.unique().tolist():
    fecha = str(fecha)
    try:
        # Intenta analizar la fecha
        datetime.datetime.strptime(fecha, '%Y-%m-%d %H:%M:%S')
        fechas_correctas.append(fecha)
    except ValueError:
        # Captura las excepciones ValueError (formato incorrecto)
        fechas_incorrectas.append(fecha)

print("Fechas correctas:")
for fecha in fechas_correctas:
    print(fecha)

print("\nFechas incorrectas:")
for fecha in fechas_incorrectas:
    print(fecha)

Fechas correctas:
2016-01-01 00:00:00
2016-01-02 00:00:00
2016-01-03 00:00:00
2016-02-01 00:00:00
2016-01-24 00:00:00
2016-01-26 00:00:00
2016-01-29 00:00:00
2016-02-08 00:00:00
2016-02-10 00:00:00
2016-02-14 00:00:00
2016-02-16 00:00:00
2016-02-17 00:00:00
2016-03-02 00:00:00
2016-02-28 00:00:00
2016-03-04 00:00:00
2016-03-12 00:00:00
2016-03-13 00:00:00
2016-03-14 00:00:00
2016-03-19 00:00:00
2016-03-21 00:00:00
2016-03-23 00:00:00
2016-03-29 00:00:00
2016-03-30 00:00:00
2016-03-31 00:00:00
2016-04-15 00:00:00
2016-04-19 00:00:00
2016-04-20 00:00:00
2016-04-22 00:00:00
2016-04-25 00:00:00
2016-04-28 00:00:00
2016-04-26 00:00:00
2016-05-08 00:00:00
2016-05-26 00:00:00
2016-05-20 00:00:00
2016-05-23 00:00:00
2016-06-12 00:00:00
2016-06-13 00:00:00
2016-07-05 00:00:00
2016-06-29 00:00:00
2016-07-08 00:00:00
2016-07-01 00:00:00
2016-07-03 00:00:00
2016-07-14 00:00:00
2016-07-15 00:00:00
2016-07-19 00:00:00
2016-07-23 00:00:00
2016-08-12 00:00:00
2016-08-03 00:00:00
2016-08-15 00:00:00
20

Vemos que tenemos solo una fecha incorrecta asi que la imputamos manualmente.

In [33]:
victimas.loc[victimas['Fecha_fallecimiento'] == '26/03/2019', 'Fecha_fallecimiento'] = datetime.datetime.strptime('26/03/2019', '%d/%m/%Y')

In [34]:
cols_a_eliminar = [
    'Aaaa',
    'Mm',
    'Dd',
] # Eliminamos estas columnas ya que tenemos las fechas correctas

victimas = victimas.drop(columns=cols_a_eliminar)

In [35]:
a_min = ['Rol', 'Victima', 'Sexo'] # Elegimos las columnas a pasar a minusculas

In [36]:
for columna in a_min:
    victimas[columna] = victimas[columna].str.lower() # Las pasamos a minusculas

In [37]:
victimas.head() # Vemos como esta conformado nuestro dataset parcialmente limpio

,Id_hecho,Fecha,Rol,Victima,Sexo,Edad,Fecha_fallecimiento
0,2016-0001,2016-01-01,conductor,moto,masculino,19,2016-01-01 00:00:00
1,2016-0002,2016-01-02,conductor,auto,masculino,70,2016-01-02 00:00:00
2,2016-0003,2016-01-03,conductor,moto,masculino,30,2016-01-03 00:00:00
3,2016-0004,2016-01-10,conductor,moto,masculino,18,SD
4,2016-0005,2016-01-21,conductor,moto,masculino,29,2016-02-01 00:00:00


In [38]:
for col in victimas.columns.tolist():
    victimas[col]=victimas[col].apply(lambda x: sd_to_null(x)) # Aplicamos la funcion de los sin dato anterior

In [39]:
victimas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Id_hecho             717 non-null    object        
 1   Fecha                717 non-null    datetime64[ns]
 2   Rol                  706 non-null    object        
 3   Victima              708 non-null    object        
 4   Sexo                 711 non-null    object        
 5   Edad                 664 non-null    float64       
 6   Fecha_fallecimiento  649 non-null    datetime64[ns]
dtypes: datetime64[ns](2), float64(1), object(4)
memory usage: 39.3+ KB


Vemos que si tenemos nulos

## Merge

In [40]:
merge = pd.merge(left=hechos, right=victimas,on='Id_hecho', how='inner') # Hacemos un merge entero

In [41]:
merge.head(1) # Vemos un registro ejemplo

,Id_hecho,Victimas,Fecha_x,Hora,Tipo_calle,Cruce,Direccion,Comuna,Longitud,Latitud,Tipo_victima,Tipo_acusado,Fecha_y,Rol,Victima,Sexo,Edad,Fecha_fallecimiento
0,2016-0001,1,2016-01-01,4,avenida,"Fernandez de la cruz, f., gral. av.","Piedra buena av. y fernandez de la cruz, f., g...",8,-58.47534,-34.68757,moto,auto,2016-01-01,conductor,moto,masculino,19.0,2016-01-01


Vemos que al hacer el merge, no se joinea la columna fecha, y se crean dos diferentes, eso se debe a q tienen valores diferentes, analicemoslas.

In [42]:
x = merge.Fecha_x.tolist() # Extraigo las fechas de la primera columna
y = merge.Fecha_y.tolist() # Extraigo las fechas de la segunda columna
for i in range(len(x)): # Por cada fecha
    if x[i] == y[i]: # Si las fechas coinciden
        pass 
    else:
        print('Fecha_hechos: ',x[i],'Fecha_victimas: ',y[i], i) # Si no coincide, muestro las diferencias

Fecha_hechos:  2016-07-02 00:00:00 Fecha_victimas:  2016-07-03 00:00:00 65
Fecha_hechos:  2017-05-24 00:00:00 Fecha_victimas:  2017-05-29 00:00:00 204


Tenemos dos fechas diferentes asi que las imputo manualmente a un valor de los dos.

In [43]:
merge.loc[merge['Fecha_x'] == '2016-07-02 00:00:00', 'Fecha_x'] = '2016-07-03 00:00:00'
merge.loc[merge['Fecha_x'] == '2017-05-24 00:00:00', 'Fecha_x'] = '2017-05-29 00:00:00'
# Cambio las fechas a las de la otra
merge.drop('Fecha_y',axis=1,inplace=True) # Y elimino la fecha 'y'
merge.rename(columns={'Fecha_x':'Fecha'}, inplace=True) # Y cambiamos el nombre de la 'x'

In [44]:
for col in victimas.columns.tolist():
    print(col,":",victimas[col].duplicated().unique())

Id_hecho : [False  True]
Fecha : [False  True]
Rol : [False  True]
Victima : [False  True]
Sexo : [False  True]
Edad : [False  True]
Fecha_fallecimiento : [False  True]


uhh problemitas, no puede haber ids duplicados.

In [45]:
victimas[victimas['Id_hecho'].duplicated()]

,Id_hecho,Fecha,Rol,Victima,Sexo,Edad,Fecha_fallecimiento
30,2016-0041,2016-03-29,pasajero_acompañante,moto,masculino,NaN,2016-03-30
99,2016-0126,2016-09-18,pasajero_acompañante,auto,masculino,60.0,NaT
164,2017-0026,2017-02-26,conductor,auto,masculino,19.0,2017-02-26
174,2017-0035,2017-03-23,pasajero_acompañante,auto,masculino,32.0,2017-03-23
175,2017-0035,2017-03-23,pasajero_acompañante,auto,masculino,30.0,2017-03-23
177,2017-0036,2017-03-29,conductor,moto,masculino,20.0,2017-03-29
187,2017-0050,2017-04-28,pasajero_acompañante,moto,masculino,16.0,2017-04-28
238,2017-0108,2017-09-02,pasajero_acompañante,auto,masculino,10.0,2017-09-02
243,2017-0112,2017-09-10,pasajero_acompañante,auto,masculino,NaN,2017-09-13
254,2017-0126,2017-10-14,conductor,auto,masculino,36.0,2017-10-14


Vemos que el id no es un duplicado realmente. Lo que sucede es que los primeros numeros son iguales entonces los toma como duplicados aunque no lo sean realmente.

Tenemos nuestros datasets sin duplicados y nuestro merge tambien.

In [46]:
# Reseteamos indices
hechos.reset_index(drop=True, inplace=True)
victimas.reset_index(drop=True, inplace=True)
merge.reset_index(drop=True, inplace=True)

In [47]:
merge.drop('Victima',axis=1,inplace=True) #  Eliminamos la columna Victima

Exportamos cada archivo como .csv

In [48]:
victimas.to_csv('../Data/victimas.csv', index=False)
hechos.to_csv('../Data/hechos.csv', index=False)
merge.to_csv('../Data/merge.csv', index=False)